In [5]:
import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
import pandas as pd
import scipy.linalg as la
from schnetpack.datasets import QM9

# Load best model where it was stored after training

best_model = torch.load("./trained_models/qm911/best_model", map_location=torch.device('cpu'))

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file 
train, val, test = spk.data.train_test_split(qm9data, split_file='./trained_models/qm911/split.npz')

#set up device and atoms converter for input

device = 'cpu'

converter = spk.data.AtomsConverter(device=device)

test_loader = spk.AtomsLoader(test, batch_size=100)
converter = spk.data.AtomsConverter(device=device)

at, props = qm9data.get_properties(idx=0)

calculator = spk.interfaces.SpkCalculator(model=best_model, device=device, energy=QM9.U0)
at.set_calculator(calculator)


print('Prediction:', at.get_total_energy())


Prediction: -1103.222900390625


In [6]:
#Extract embedding from schnetpack
from schnetpack.representation import SchNet

model = SchNet()
layer_output=None

def layer_hook_fn(self, inp_tensor, out_tensor):
    global layer_output
    layer_output=out_tensor
    
model.embedding.register_forward_hook(layer_hook_fn)

print(layer_output)







None


In [ ]:




#activation = {}

#inputs = converter(at)

#def get_activation(name):
#    def hook(module, input, output):
#        activation[name] = output.detach()
#    return hook

#model = SchNet()
#model.embedding.register_forward_hook(get_activation('Embedding'))
#output = model(inputs)

#print(activation['Embedding'])
    